## Install neccessary packages

In [ ]:
# Python 3.11.13
!python --version

Python 3.11.13


In [ ]:
%pip install -r ./extract_text/requirements.txt
%pip install -r ./text_normalizer_vietnamese/requirements.txt
%pip install pymupdf
%pip install matplotlib pandas

In [5]:
%pip install -r ./sentence_alignment/requirements.txt

  Using cached numba-0.60.0-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
  Using cached googletrans-4.0.0rc1-py3-none-any.whl
  Using cached sentence_splitter-1.4-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached sentence_transformers-3.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached llvmlite-0.43.0-cp311-cp311-win_amd64.whl.metadata (4.9 kB)
  Using cached numpy-2.0.2-cp311-cp311-win_amd64.whl.metadata (59 kB)
  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached hstspreload-2025.1.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached h2-3.2.0-py2.py3-none-any.whl.metada

## 1. Lấy dữ liệu văn bản từ tệp PDF

In [ ]:
# VĂN BẢN CHỮ QUỐC NGỮ
# !python ./pdf_to_text/main.py -p ./data/source/Book-Tay_Du_Ky-Viet.pdf -l viet -o ./data/raw_text/vietnamese -fp 48 -np 2 --rule_path ./pdf_to_text/removal_rules_vietnamese.json --use_rule
# Trang chứa nội dung: 48 -> 3415

In [101]:
# VĂN BẢN CHỮ HÁN
# !python ./pdf_to_text/main.py -p ./data/source/Book-Tay_Du_Ky-Trung.pdf -l trung -o ./data/raw_text/chinese -fp 1 -np 2 --rule_path ./pdf_to_text/removal_rules_chinese.json --use_rule --use_page_split
!python ./extract_text/extract_text_from_pdf.py
# Trang chứa nội dung: 1 -> 542

## 2. Chuẩn hóa dữ liệu văn bản 

### 2.1 Chuẩn hóa văn bản chữ Quốc ngữ

In [ ]:
# !python ./text_normalizer_vietnamese/main.py ./data/raw_text/vietnamese/ ./data/cleaned_text/vietnamese

### 2.2 Chuẩn hóa văn bản chữ Hán

In [ ]:
# !python ./text_normalizer_chinese/main.py -i ./data/raw_text/chinese/ -o ./data/cleaned_text/chinese

In [17]:
!python ./sentence_alignment/sentence_alignment.py

Source language: Chinese, Number of sentences: 10
Target language: Vietnamese, Number of sentences: 13
Embedding source and target text using paraphrase-multilingual-mpnet-base-v2 ...
Performing first-step alignment ...
Performing second-step alignment ...
Finished! Successfully aligning 10 Chinese sentences to 13 Vietnamese sentences

[([np.int64(0)], [np.int64(0)]), ([np.int64(1)], [np.int64(1)]), ([np.int64(2)], [np.int64(2)]), ([np.int64(3)], [np.int64(3)]), ([np.int64(4)], [np.int64(4)]), ([np.int64(5)], [np.int64(5)]), ([np.int64(6)], [np.int64(6)]), ([], [np.int64(7)]), ([np.int64(7)], [np.int64(8)]), ([np.int64(8)], [np.int64(9)]), ([np.int64(9)], [np.int64(10)]), ([], [np.int64(11)]), ([], [12])]
TDK_001.001.001.01:
蓋聞天地之數,有十二萬九千六百歲為一元.
Từng nghe số của trời đất, gồm một trăm hai mươi chín nghìn sáu trăm năm là một nguyên.

TDK_001.001.001.02:
將一元分為十二會,乃子、丑、寅 、卯、辰、巳、午、未、申、酉、戌、亥之十二支也.
Một nguyên chia làm mười hai hội, tức mười hai chi: Tý, Sửu, Dần, Mão, Thìn, Tỵ, Ngọ, Mùi, Thâ